In [1]:
print("""
@Description: 张量元数据：大小、偏移量和步长
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-17 22:24:58
""")


@Description: 张量元数据：大小、偏移量和步长
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-17 22:24:58



In [2]:
import torch

## 另一个张量的存储视图

In [4]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1]
second_points.size()

torch.Size([2])

得到的张量在存储区中的偏移量为 2，这是因为我们需要跳过第 1 个点，该点有两个元素。同时函数 size()是 Size 类的一个实例，因为该张量是一维的，所以它包含一个元素。需要重点注意的是，函数 size()所包含的信息与张量对象的 shape 属性所包含的信息是一样的。

In [5]:
second_points.shape

torch.Size([2])

In [6]:
second_points.storage_offset()

2

In [7]:
second_points.stride()

(1,)

重要的是，正如我们预期的那样，子张量的维度少了一维，同时仍然索引了与原始张量 points 相同的存储区。这也意味着更改子张量会对原始张量产生影响。

In [8]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1]
second_points[0] = 999.
points

tensor([[  4.,   1.],
        [999.,   3.],
        [  2.,   1.]])

In [9]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_points = points[1].clone()
second_points[0] = 999
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

### 无复制转置

In [44]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [45]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

很容易地验证这 2 个张量共享同一个存储区：❌

**似乎因为一些原因无法等到相等的判断**

In [46]:
id(points.storage()) == id(points_t.storage())

False

In [51]:
print(id(points.storage()), id(points_t.storage()))

2411760765392 2411760765248


In [56]:
assert points.storage().data_ptr() == points_t.storage().data_ptr()

In [57]:
print(points.storage().data_ptr(), points_t.storage().data_ptr())

2411737606528 2411737606528


### 高维转置

In [59]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [60]:
transpose_t.shape

torch.Size([5, 4, 3])

In [61]:
some_t.stride()

(20, 5, 1)

In [62]:
transpose_t.stride()

(1, 5, 20)